In [26]:
import tensorflow as tf
import numpy as np

m1 = [[1.,2.],[3.,4.]]
m2 = np.array([[1.,2.],[3.,4.]])
m3 = tf.constant([[1.,2.],[3.,4.]])

print(type(m1))
print(type(m2))
print(type(m3))

t1 = tf.convert_to_tensor(m1)
t2 = tf.convert_to_tensor(m2)
t3 = tf.convert_to_tensor(m3)

print(type(t1))
print(type(t2))
print(type(t3))

<class 'list'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [39]:

m1 = tf.constant([[1.,2.]])
m2 = tf.constant([[1.],[2.]])
m3 = tf.constant([ [[1,2],
                    [3,4],
                    [5,6]],
                   [[7,8],
                    [9,10],
                    [11,12]] ])

print(m1)
print(m2)
print(m3)

Tensor("Const_74:0", shape=(1, 2), dtype=float32)
Tensor("Const_75:0", shape=(2, 1), dtype=float32)
Tensor("Const_76:0", shape=(2, 3, 2), dtype=int32)


In [35]:
x = tf.constant([[1,2]])
negMatrix = tf.negative(x)
print(negMatrix)

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(negMatrix)
print(result)

Tensor("Neg_13:0", shape=(1, 2), dtype=int32)
[[-1 -2]]


In [25]:
sess = tf.InteractiveSession()

x = tf.constant([[1,2]])
negMatrix = tf.negative(x)
print(negMatrix)

print(negMatrix.eval())

sess.close()

Tensor("Neg_4:0", shape=(1, 2), dtype=int32)
[[-1 -2]]


In [57]:
sess = tf.InteractiveSession()

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spike = tf.Variable(False)
spike.initializer.run()

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i - 1] > 5:
        tf.assign(spike, True).eval()
    else:    
        tf.assign(spike, False).eval()

    print("Spike(%d) %r" % (raw_data[i] - raw_data[i - 1], spike.eval()))        

sess.close()

Spike(1) True
Spike(6) False
Spike(-9) True
Spike(1) True
Spike(5) False
Spike(0) True
Spike(7) False


In [86]:
sess = tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spikes = tf.Variable([False] * len(raw_data), name = 'spikes')
#spikes.initializer.run()
init = tf.global_variables_initializer()

saver = tf.train.Saver()

sess.run(init)

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i - 1] > 5:
        spikes_val = spikes.eval()
        spikes_val[i] = True
        tf.assign(spikes, spikes_val).eval()

save_path = saver.save(sess, "spikes.ckpt")
print("spikes data saved in file %s" % save_path)

sess.close()

spikes data saved in file spikes.ckpt


In [85]:
tf.reset_default_graph()

sess = tf.InteractiveSession()
spikes = tf.Variable([False] * len(raw_data), name = 'spikes')

saver = tf.train.Saver()

saver.restore(sess, "spikes.ckpt")

print(spikes.eval())

sess.close()

INFO:tensorflow:Restoring parameters from spikes.ckpt
[False False False False False False False False]


In [96]:
tf.reset_default_graph()

raw_data = np.random.normal(10, 1, 100)

alpha = tf.constant(0.05)
curr_value = tf.placeholder(tf.float32)
prev_avg = tf.Variable(0.)
update_avg = alpha * curr_value + (1. - alpha) * prev_avg

avg_hist = tf.summary.scalar("running_average", update_avg)
value_hist = tf.summary.scalar("incoming_value", curr_value)
merge = tf.summary.merge_all()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    writer = tf.summary.FileWriter("./logs", sess.graph)
    sess.run(init)
    #sess.add_graph(sess.graph) #визуализируем граф
    for i in range(len(raw_data)):
        summary_str,curr_avg = sess.run([merge, update_avg], feed_dict = {curr_value: raw_data[i]})
        sess.run(tf.assign(prev_avg, curr_avg))
        print(raw_data[i], curr_avg)
        writer.add_summary(summary_str, i)

10.2014650488 0.510073
8.11361773747 0.89025
10.8675391342 1.38911
9.99003082891 1.81916
10.1647291117 2.23644
8.80950308463 2.56509
9.90191107955 2.93193
9.29839144086 3.25026
10.80461728 3.62797
10.3847644968 3.96581
10.0625397872 4.27065
9.7825594574 4.54625
11.0562976164 4.87175
10.542279163 5.15527
9.28425484879 5.36172
9.83976112421 5.58563
10.6391742535 5.8383
11.5325831039 6.12302
9.44670372755 6.2892
11.2490847944 6.53719
10.3393161468 6.7273
9.70127257144 6.876
10.917056378 7.07805
11.5590691817 7.3021
10.2938216011 7.45169
7.60298924737 7.45925
11.0722303444 7.6399
10.756025536 7.79571
10.8633643722 7.94909
9.82751519197 8.04301
10.0029319383 8.14101
10.2464961673 8.24628
11.3991159999 8.40392
12.1906412297 8.59326
10.5130236602 8.68925
8.84522825996 8.69705
9.79059216636 8.75172
9.98000858902 8.81314
7.45754374686 8.74536
11.9115654238 8.90367
9.07187511514 8.91208
10.2814351056 8.98055
10.9920008678 9.08112
10.7023458912 9.16218
10.8044240739 9.24429
9.48030471243 9.25609
